In [51]:
from binance.spot import Spot as sp
import inspect as i
import json
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pytz
from plotly.subplots import make_subplots
from finta import TA as ta

tz_local = pytz.timezone("Europe/Zurich")

apiKey = "API KEY"
apiKey_secret = "API KEY"

baseURL = "https://testnet.binance.vision"

pd.options.display.float_format = "{:,.3f}".format

In [52]:
#API SETUP
sp_client = sp(key=apiKey, secret=apiKey_secret)

#get Different cryptos
dict_symbols = sp_client.exchange_info()

#API CALL
api_interval = "1m"
pd_interval = "T"
api_limit = 1000
end_time = dt.datetime.now()
end_time_api = int(round(end_time.timestamp()*1000,0))

dict_data = sp_client.klines(endTime=end_time_api, symbol="BTCEUR", interval="1m", limit=api_limit)

In [53]:
df_symbols = pd.json_normalize(dict_symbols["symbols"])

In [54]:
headers_df = ["openTime","open", "high","low","close","vol","closeTime","quiteAssetVolume","trades_qty","baseAssetVol_taker","baseQuoteVol_taker","ignore"]

df_data_raw = pd.DataFrame(dict_data, columns=headers_df)
#convert to datetimes
df_data_raw["openTime"] = pd.to_datetime(df_data_raw["openTime"]/1000, unit="s")
df_data_raw["closeTime"] = pd.to_datetime(df_data_raw["closeTime"]/1000, unit="s")

#round to nearest second
df_data_raw["closeTime"] = df_data_raw["closeTime"].dt.floor("S")

#change timezone

#reset index to opentime
df_data_raw.reset_index(inplace=True,drop=True)
df_data_raw.set_index("openTime", inplace=True)

# convert all none-date columns to float
cols = df_data_raw.select_dtypes(exclude=["datetime64"]).columns
df_data_raw[cols] = df_data_raw[cols].apply(pd.to_numeric, downcast="float",errors="coerce")

# mark TZ as UTC then change to CET
df_data_raw.index = df_data_raw.index.tz_localize("UTC").tz_convert("CET")
df_data_raw["closeTime"] = df_data_raw["closeTime"].dt.tz_localize("UTC").dt.tz_convert("CET")


In [55]:
df_data_raw.tail(1)

,open,high,low,close,vol,closeTime,quiteAssetVolume,trades_qty,baseAssetVol_taker,baseQuoteVol_taker,ignore
openTime,,,,,,,,,,,
2021-09-18 10:23:00+02:00,"41,572.602","41,579.148","41,572.590","41,578.070",0.157,2021-09-18 10:23:59+02:00,"6,520.717",32.000,0.131,"5,443.917",0.000


In [56]:
df_data_raw.index.min().to_pydatetime().strftime("%Y-%m-%d %H:%M:%S")

'2021-09-17 17:44:00'

In [57]:
#CREATE BASE TIMELINE TO JOIN AGAINST
end_time_pd = df_data_raw.index.max().to_pydatetime().strftime("%Y-%m-%d %H:%M:%S")
df_base = pd.DataFrame(pd.date_range(end=end_time_pd, freq=pd_interval, periods=api_limit), columns=["dateTime"])

df_base.reset_index(drop=True, inplace=True)
df_base.set_index("dateTime", inplace=True)
df_base.index = df_base.index.tz_localize("CET")

In [58]:
#JOIN AGAINST BASE TIMELIME
df_data = pd.concat([df_base, df_data_raw], axis=1)
df_data.index.names = ["dateTime"]

#CREATE DATE SKKIP LIST
l_skipDates = df_data.loc[df_data["open"].isna() == True].index.strftime("%Y-%m-%d %H:%M:%S").tolist()



In [59]:
#date_offset=(dt.datetime.now()-dt.timedelta(hours=24))
#date_offset
#dt64 = np.datetime64(date_offset)
# 
dateOffset_now = tz_local.localize(dt.datetime.now(), is_dst=None)
offset_hrs = 48

df_data_dtFilter = df_data[df_data.index >= dateOffset_now - dt.timedelta(hours=offset_hrs)].copy()


In [60]:
fig = go.Figure(data=go.Candlestick(
    x=df_data_dtFilter.index
    ,open=df_data_dtFilter['open']
    ,close=df_data_dtFilter["close"]
    ,high=df_data_dtFilter["high"]
    ,low=df_data_dtFilter["low"]
))                             


fig.update_xaxes(rangebreaks=[dict(values=l_skipDates, dvalue=(60* 1000))]
)

fig.show()

In [61]:
df_hourly = df_data[df_data.index > "2021-08-06"].copy()
dailyClose = df_data_dtFilter[['close']]
dailyPctChange = dailyClose.pct_change()
dailyPctChange.fillna(0, inplace=True)

dailyReturns_log = np.log(dailyClose.pct_change()+1)


In [62]:
#add rolling averages

wd_short = 5
wd_medium = 20
wd_medium_ema = 13
wd_long = 200

df_close = df_data_dtFilter[["close"]].copy()
df_close["mav_short"] = df_close["close"].rolling(window=wd_short).mean()
df_close["mav_medium"] = df_close["close"].rolling(window=wd_medium).mean()
df_close["mav_long"] = df_close["close"].rolling(window=wd_long).mean()

df_close["ema_short"] = df_close["close"].ewm(span=wd_short,  ignore_na=True).mean()
df_close["ema_med"] = df_close["close"].ewm(span=wd_medium_ema,  ignore_na=True).mean()

In [63]:
# Bollinger Bands
df_close["bohlinger_upper"] = df_close["mav_medium"] + (2*(df_close["mav_medium"].rolling(window=wd_medium).std()))
df_close["bohlinger_lower"] = df_close["mav_medium"] - (2*(df_close["mav_medium"].rolling(window=wd_medium).std()))


In [64]:
# Stochastic Oscillator

so_wd_k = 14
so_wd_d = 3

df_close["k_line"] = ((df_close["close"] - df_close["close"].rolling(window=so_wd_k).min()) * 100 / 
    (df_close["close"].rolling(window=so_wd_k).max() - df_close["close"].rolling(window=so_wd_k).min()))

df_close["d_line"] = df_close["k_line"].rolling(window=so_wd_k).mean()

df_close["so_20"] = 20
df_close["so_80"] = 80

In [65]:
# MACD
macd_fast = 26
macd_slow = 12
macd_smooth = 9

df_close["macd_ema_fast"] = df_close["close"].ewm(span=macd_fast, adjust=False, ignore_na=True).mean()
df_close["macd_ema_slow"] = df_close["close"].ewm(span=macd_slow, adjust=False, ignore_na=True).mean()
df_close["macd_base"] = df_close["macd_ema_slow"] - df_close["macd_ema_fast"]
df_close["macd_signal"] = df_close["macd_base"].ewm(span=macd_smooth, adjust=False, ignore_na=True).mean()
df_close["macd_hist"] = df_close["macd_base"] - df_close["macd_signal"]
df_close["macd_color"] = np.where(df_close["macd_hist"]<0, "red", "green")

In [66]:
def comp_df(df, col1, col2, metric):
    comp = str(metric+"_comp")
    
    df[comp] = np.where(df[col1] > df[col2], 1,-1)
    return df


comp_df(df_close,"ema_short","ema_med","ema")
    


ema_conditions = [
    ((df_close["ema_short"].shift(3) > df_close["ema_med"].shift(3)) & (df_close["ema_short"].shift(4) < df_close["ema_med"].shift(4)) & (abs(df_close["ema_comp"].rolling(window=3).sum()) == 3)),
    ((df_close["ema_short"].shift(3) < df_close["ema_med"].shift(3)) & (df_close["ema_short"].shift(4) > df_close["ema_med"].shift(4)) & (abs(df_close["ema_comp"].rolling(window=3).sum()) == 3))  
]

ema_choices = [1,-1]

df_close["ema_switch"] = np.select(ema_conditions, ema_choices, default=0)

df_marks = df_close.loc[df_close["ema_switch"].isin([1,-1])]

In [67]:
def udf_slope(x):
    slope = np.polyfit(range(len(x)), x, 1)[0]
    return slope


df_close["slope_close"] = df_close["close"].rolling(4, min_periods=4).apply(udf_slope)
df_close["slope_close_ewm"] = df_close["slope_close"].ewm(span=3, ignore_na=True).mean()

In [68]:
fig2 = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01
                    , specs=[[{}],[{}],[{}],[{"secondary_y": True}]])

### ---------------> OHLC
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["close"]
        , name="CLOSE"
        , line=dict(color="black", width=2)
), row=1, col=1)         


### ---------------> ema_short
fig2.add_trace(go.Scatter(
    mode ="lines"
    , x=df_close.index
    , y=df_close["ema_short"]
    , line = dict(color="blue")
    , name="ema_{}".format(wd_short)
), row=1, col=1)

### ---------------> ema_med
fig2.add_trace(go.Scatter(
    mode ="lines"
    , x=df_close.index
    , y=df_close["ema_med"]
    , line = dict(color="yellow")
    , name="ema_{}".format(wd_medium_ema)
), row=1, col=1)


######### 3rd graph
### ---------------> BASE CLOSE

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["close"]
        , name="CLOSE"
        , line=dict(color="black", width=2)

    ), row=2, col=1
)

### ---------------> Bohlinger Up
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["bohlinger_upper"]
        , line=dict(dash="dash", color="green")
        , name="bollinger_up"
    ), row=2, col=1
)

### ---------------> Bohlinger Low
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["bohlinger_lower"]
        , line=dict(dash="dash", color="green")
        , name="bollinger_low"
    ), row=2, col=1
)

######### 4th graph
### ---------------> MACD

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["macd_base"]
        , name="MACD"
        , line=dict(color="black", width=2)


    ), row=3, col=1
)

### ---------------> MACD SIGNAL

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["macd_signal"]
        , name="macd_signal"
        , line=dict(color="lightblue", width=2, dash="dot")

    ), row=3, col=1
)

### ---------------> MACD HIST

fig2.add_trace(
    go.Bar(
        x=df_close.index
        , y=df_close["macd_hist"]
        , name="macd_hist"
        , marker_color=df_close["macd_color"]

    ), row=3, col=1
)

######### 6th graph
### ---------------> BASE CLOSE

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["close"]
        , name="CLOSE"
        , line=dict(color="black", width=2)
        , yaxis="y4"

    ), row=4, col=1
)

### ---------------> %K
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["k_line"]
        , name="%K"
        , line=dict(color="red", width=2, dash="solid")
        

    ), row=4, col=1, secondary_y=True
)

### ---------------> %D
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["d_line"]
        , name="%D"
        , line=dict(color="orange", width=2, dash="solid")
        

    ), row=4, col=1, secondary_y=True
)

### ---------------> SO bounds
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["so_20"]
        , name="bound_20"
        , line=dict(color="blue", width=0.5, dash="dash")
    ), row=4, col=1, secondary_y=True
)

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["so_80"]
        , name="bound_80"
        , line=dict(color="blue", width=0.5, dash="dash")
    ), row=4, col=1, secondary_y=True
)


rn_yaxis_a = (df_close["close"].min()) * 1.001
rn_yaxis_b = (df_close["close"].max()) * 1.001

fig2.update_layout(
    width=1500
    , height=2000
    , xaxis1 = dict(rangeslider=dict(visible=False))
    , xaxis6 = dict(rangeslider=dict(visible=True))

    , yaxis1 = dict(range=[rn_yaxis_a,rn_yaxis_b])
    , yaxis2 = dict(range=[rn_yaxis_a,rn_yaxis_b])
    , yaxis4 = dict(range=[rn_yaxis_a,rn_yaxis_b])



)


for idx, x in df_marks.iterrows():
    x_date = idx.strftime("%Y-%m-%d %H:%M:%S")
    fig2.add_shape(go.layout.Shape(type="line", xref="x", x0=x_date, x1=x_date, y0=-10000, y1=100000, line={"color":("green" if x["ema_switch"] == 1 else "red"), "width":2, "dash":"dot"}), row=[1,2,4], col="all")


fig2.update_xaxes(matches="x"
    ,rangebreaks=[dict(values=l_skipDates, dvalue=(60* 1000))]
)

fig2.show()

In [69]:
df_close[-100:].to_clipboard(excel=True)

In [70]:
df_close.tail(1)

,close,mav_short,mav_medium,mav_long,ema_short,ema_med,bohlinger_upper,bohlinger_lower,k_line,d_line,...,macd_ema_fast,macd_ema_slow,macd_base,macd_signal,macd_hist,macd_color,ema_comp,ema_switch,slope_close,slope_close_ewm
dateTime,,,,,,,,,,,,,,,,,,,,,
2021-09-18 10:23:00+02:00,"41,578.070","41,576.164","41,591.544","41,501.291","41,578.372","41,580.848","41,646.706","41,536.381",35.991,33.004,...,"41,572.700","41,580.958",8.258,13.307,-5.048,red,-1,0,-1.504,0.114


In [71]:
from IPython import display
import time
import random as rnd

fig3 = go.Figure()

np.random.seed(1111)
data = np.random.randint(1,high=100, size=30)
dft = pd.DataFrame({"idx": pd.date_range("2021-09-05", periods=30, freq="H"), "data":data})
dft.set_index("idx", inplace=True)

print(dft.index[-1])
print([dft.index[-1] + dt.timedelta(hours=1), rnd.randint(1,100)])


# for i in range (0,10):

#     fig3.data=[]

#     fig3.add_trace(
#         go.Scatter(
#             mode="lines"
#             , x=dft.index
#             , y=dft["data"]
#             , name="CLOSE"
#             , line=dict(color="black", width=2)
#     ))

    

#     fig3.add_trace(
#         go.Scatter(
#             mode="markers"
#             , x=[dft.index[-1]]
#             , y=[dft.iloc[len(dft)-1]["data"]]
#             , name="LAST"
#             , line=dict(color="RED")
#             , marker=dict(size=20)
#     ))

#     display.display(fig3.show())
#     display.clear_output(wait=True)
#     time.sleep(0.1)

#     dft.loc[dft.index[-1] + dt.timedelta(hours=1)] = [rnd.randint(1,100)]




2021-09-06 05:00:00
[Timestamp('2021-09-06 06:00:00'), 9]


In [72]:
so_comp_cond=[
    ((df_close["k_line"].shift(1) < 20) & (df_close["k_line"] > df_close["d_line"]))
    , ((df_close["k_line"].shift(1) > 80) & (df_close["k_line"] < 80))
]

so_comp_choices =[1,-1]


macd_comp_cond=[
    ((df_close["macd_base"].shift(1) < df_close["macd_signal"].shift(1)) & (df_close["macd_base"] > df_close["macd_signal"]))
    , ((df_close["macd_base"].shift(1) > df_close["macd_signal"].shift(1)) & (df_close["macd_base"] < df_close["macd_signal"]))
]

macd_comp_choices =[1,-1]




df_close["so_switch"] = np.select(so_comp_cond, so_comp_choices, default=0)
df_close["macd_switch"] = np.select(macd_comp_cond, macd_comp_choices, default=0)


df_close.loc[df_close["macd_switch"] != 0]

df_close["d_line"].tail(1).item()

33.00429680030055

In [73]:
import secrets

df_book = pd.DataFrame(columns= ["tradeType","buyPrice","qty","buyTime","sellPrice", "sellTime", "profit", "profit_per_second", "status"])
funds = 100

openTrade_so = 0
trade_id_so = ""

def udf_trade(openTrade, ttype, price_buy):
    global funds
    if funds == 0:
        return print("ERROR: Not sufficient funds")
    
    if openTrade == 0 and ttype == "b":
        print("Buying...")
        tradeid = str(secrets.token_hex(5))

        while tradeid in df_book.index:
            tradeid = str(secrets.token_hex(5))

    else: 
        return print("invalid trade combination")

    print(f"Trade ID found, starting Trade: {str(tradeid)}")

    # assumes investment of full funds
    df_book.loc[str(tradeid), ["buyPrice","qty", "buyTime", "status"]] = [price_buy, (funds/price_buy), time.time(), "bought"]
    print("Purchase Complete")
    openTrade = 1
    print(df_book)
    funds = 0
    return openTrade, tradeid


price_buy = df_close["close"].tail(1).item()

openTrade_so, trade_id_so = udf_trade(openTrade_so, "b", price_buy)
print(f"[[ buy complete: {openTrade_so} > {trade_id_so}]]")



Buying...
Trade ID found, starting Trade: f8ea47bbae
Purchase Complete
           tradeType   buyPrice   qty           buyTime sellPrice sellTime  \
f8ea47bbae       NaN 41,578.070 0.002 1,631,953,426.166       NaN      NaN   

           profit profit_per_second  status  
f8ea47bbae    NaN               NaN  bought  
[[ buy complete: 1 > f8ea47bbae]]


In [74]:
df_close.at[df_close["close"].tail(1).index,"close"] = 1
df_close.tail(1)

,close,mav_short,mav_medium,mav_long,ema_short,ema_med,bohlinger_upper,bohlinger_lower,k_line,d_line,...,macd_base,macd_signal,macd_hist,macd_color,ema_comp,ema_switch,slope_close,slope_close_ewm,so_switch,macd_switch
dateTime,,,,,,,,,,,,,,,,,,,,,
2021-09-18 10:23:00+02:00,1.000,"41,576.164","41,591.544","41,501.291","41,578.372","41,580.848","41,646.706","41,536.381",35.991,33.004,...,8.258,13.307,-5.048,red,-1,0,-1.504,0.114,1,0


In [75]:
import multiprocessing as mp

mp.cpu_count()

16